In [1]:
import pandas as pd
import numpy as np
import nest_asyncio
import asyncio
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

from utils import read_pcap_file, combine_request_reply

nest_asyncio.apply()  # Needed for running the async function in the notebook


pd.set_option('display.max_colwidth', None)  # No truncation of column content

# PCAP files

In [2]:
dir_contents = [i for i in os.listdir("./data") if ".pcap" in i]
dir_contents

['sessionfailureEMS.pcap',
 'EMSSessionNoResp.pcap',
 'EMSSession11.pcap',
 'EMSSecurityAccessFailure.pcap',
 'PreconditionCheckFailA2.pcap',
 'Endless78.pcap',
 'vinfailure_continue.pcap',
 'EMSSession14.pcap',
 'PreconditionCheckNegResp.pcap',
 'EMSSession12.pcap',
 'EMSSession13.pcap',
 'vinfailure_stop.pcap']

# Read PCAP, Convert DataFrame

In [3]:
pcap_file = 'EMSSecurityAccessFailure.pcap'

file_path = os.path.join("data", pcap_file)

df = asyncio.run(read_pcap_file(file_path))

df.head()


,number,timestamp,source,target,request,sid,error
0,10,2024-09-18 11:35:32,0x0e00,0x1032,True,0x3E,N/A
1,12,2024-09-18 11:35:32,0x1032,0x0e00,False,0x7E,N/A
2,14,2024-09-18 11:35:32,0x0e00,0x1032,True,0x3E,N/A
3,16,2024-09-18 11:35:32,0x1032,0x0e00,False,0x7E,N/A
4,18,2024-09-18 11:35:32,0x0e00,0x1032,True,0x22,N/A


# Merge Request and Reply into One Row

In [4]:
reply_request = combine_request_reply(df)
reply_request

,request_sid,reply_sid,error
0,0x3E,0x7E,N/A
1,0x3E,0x7E,N/A
2,0x22,0x62,N/A
3,0x10,0x50,N/A
4,0x3E,0x7E,N/A
5,0x3E,0x7E,N/A
6,0x22,0x62,N/A
7,0x10,0x67,N/A
8,0x27,0x7F,0x22


# Initialize LLM

In [5]:
# Load environment variables from .env file
load_dotenv()

# Get the OPENAI_API_KEY from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(api_key=openai_api_key, temperature=0, model="gpt-4o")

In [8]:
template = """ The following is a DataFrame of paired request and reply messages (and possible errors) from a vehicle's 
Electronic Control Unit (ECU) system connected to a diagnostic tool. Each row represents a message sent from the 
diagnostic tool and the ECU's response. These messages are coded in the Unified Diagnostic Services (UDS) protocol.

Using your understanding of UDS SID and error codes, create a concise summary (max. 25 words) of what's happening and any potential 
errors with the system.

Messages: {messages}
"""

prompt = PromptTemplate(
    input_variables=["messages"],
    template=template,
)

chain = prompt | llm

result = chain.invoke({"messages": reply_request.to_string()}).content

print("\nSummary: \n", result)


Summary: 
 The ECU system is mostly responsive, but a security access request (0x27) failed with a "Conditions Not Correct" error (0x7F 0x22).
